In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

c:\Users\TEJKIRAN\anaconda3\envs\langgraph\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

Simple AI Assistant

In [7]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

I am Gemma, an open-weights AI assistant. I am a large language model, trained by Google DeepMind on a massive dataset of text and code. My purpose is to help users by understanding and responding to their requests in a helpful, informative, and comprehensive way.

Here are some key things to know about me:

* **Open-weights:** My weights are publicly accessible, meaning anyone can see and use the underlying code that makes me work. This promotes transparency and collaboration in the AI community.
* **Text-based:** I operate solely through text. I can read and understand your written questions and requests, and I generate my responses as text.
* **Knowledge cutoff:** My knowledge is based on the data I was trained on, which has a specific cutoff point. I don't have access to real-time information or the ability to search the internet for updates.
* **No tools:** I am a purely language-based model. I cannot interact with external tools or software.

I'm still under development, but I'm 

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [8]:
store={}

In [9]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [10]:
config = {"configurable": {"session_id": "firstchat"}}

In [11]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [12]:
model_with_memory.invoke(("Hi! I'm Tejkiran"),config=config).content

"Hi Tejkiran! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"

In [13]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Tejkiran. 😄 \n\nI remember that you told me earlier!\n'

In [14]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Tejkiran"), AIMessage(content="Hi Tejkiran! It's nice to meet you. 👋\n\nWhat can I do for you today?\n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 17, 'total_tokens': 43, 'completion_time': 0.047272727, 'prompt_time': 0.000101179, 'queue_time': 0.01421483, 'total_time': 0.047373906}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5bbcfcb6-93e5-4d7e-b0eb-5a1c9d3e5d43-0', usage_metadata={'input_tokens': 17, 'output_tokens': 26, 'total_tokens': 43}), HumanMessage(content='tell me what is my name?'), AIMessage(content='Your name is Tejkiran. 😄 \n\nI remember that you told me earlier!\n', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 58, 'total_tokens': 79, 'completion_time': 0.038181818, 'prompt_time': 0.001828613, 'queue_time': 0.013477427, 'total_time': 0.040010431}, 'mo

RAG with LCEL

In [15]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [10]:
### Reading the txt files from source directory
loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings
'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB
db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [12]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [13]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Llama 3 is an 8B parameter language model released in April 2024. 

Here are three important points:

* **Parameter Count:** Llama 3 has 8 billion parameters, indicating its size and complexity.
* **Release Date:** It was released in April 2024, making it a relatively recent development in the field of AI.
* **Accessibility:**  Versions of Llama 3 have been made accessible outside of Meta's internal use, suggesting a commitment to open-source development and wider adoption. 






Let's Start with Tools and Agents

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [17]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [18]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [19]:
tool.name

'wikipedia'

In [20]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [21]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [22]:
tool.return_direct

False

In [23]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [24]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [25]:
class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [26]:
tool=WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [25]:
tool.name

'wiki-tool'

In [26]:
tool.description

'look up things in wikipedia'

In [27]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up in Wikipedia, should be 3 or less words',
  'type': 'string'}}

In [28]:
tool.return_direct

True

In [29]:
print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


youtube search tool

In [27]:
from langchain_community.tools import YouTubeSearchTool

tool=YouTubeSearchTool()

In [28]:
tool.run("money pechu")

"['https://www.youtube.com/watch?v=EMlCVJlUAfo&pp=ygULbW9uZXkgcGVjaHU%3D', 'https://www.youtube.com/watch?v=kQLwtyHinCk&pp=ygULbW9uZXkgcGVjaHU%3D']"

In [29]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()

In [30]:
tool.invoke({"query": "What is the latest news about zoho founder"})

[{'url': 'https://www.newsbytesapp.com/news/business/zoho-founder-sridhar-vembu-criticizes-cash-rich-tech-giants-for-layoffs/story',
  'content': "Zoho's founder, Vembu, has criticized Freshworks for laying off around 660 employees despite reporting a 22% growth in revenue and approving a $400 million share buyback program."},
 {'url': 'https://www.zoho.com/inthenews-detail.html',
  'content': "Zoho Looks To Tap Digital Agencies With New Edition Of Zoho Social. Zoho Founder Sridhar Vembu On Zoho's 21 Years, The Culture, Anonymous Open Houses And More. Zoho's New Operating System Zoho One Bundles All Its 35+ Apps, Costs Just $1 A Day Per User. Zoho Founder Sridhar Vembu On The State Of Saas, On What Inspires Him And More"},
 {'url': 'https://en.wikipedia.org/wiki/Sridhar_Vembu',
  'content': 'The employee stated that Vembu was hostile to him for criticising the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist paramilitary organisation.[17][18]\nIn January 2020, Vembu met with crit

In [ ]:
from langchain.agents import AgentType, load_tools, initialize_agent

In [32]:
tool = load_tools(['wikipedia'],llm=llm)

In [33]:
agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

C:\Users\TEJKIRAN\AppData\Local\Temp\ipykernel_5752\2451433261.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [35]:
agent.run("What is the current GDP of india?")



> Entering new AgentExecutor chain...
Thought: I need to look up India's GDP on Wikipedia. 
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 141th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. Since

'The current nominal GDP of India is the fifth-largest in the world.'